In [17]:
import numpy as np

N = 10
p = 0.4


def matrix(N):
  P = np.zeros((N + 1, N + 1))
  for i in range(N):
      P[i, i + 1] = p  # Probability of winning a round
      P[i, i - 1] = 1 - p  # Probability of losing a round
  P[0, 0] = 1  # Staying at 0 when you're already at 0
  P[N, N] = 1  # Staying at N when you're already at N
  return P

P=matrix(N)
e_val, e_vec = np.linalg.eig(P.T)  # Transpose P for right e_vec

# Find the eigenvector corresponding to eigenvalue 1
stat_dbt = np.real(e_vec[:, np.argmax(np.abs(e_val))])

# Normalize the stationary distribution so that it sums to 1
stat_dbt /= np.sum(stat_dbt)

print(stat_dbt)


[2.39999634e-01 9.59997070e-02 3.83995801e-02 1.53593040e-02
 6.14283222e-03 2.45564682e-03 9.79780456e-04 3.87781128e-04
 1.48227125e-04 4.78152158e-05 6.00079692e-01]


In [18]:

r_val = [0.1, 0.01, 1e-3, 1e-4]

# Create the transition matrix with restart probability
def crt_trns_mat(N, p, r):
    P = matrix(N)
    # Add restart probability
    P = (1 - r) * P + r / (N + 1)
    return P

for r in r_val:
    # Create the transition matrix with restart probability
    P = crt_trns_mat(N, p, r)

    # Use the power method to compute PageRank
    num_iter = 1000
    init_page_rank = np.ones(N + 1) / (N + 1)  # Start with a uniform distribution

    page_rank = init_page_rank
    for _ in range(num_iter):
        page_rank = np.dot(P.T, page_rank)

    print(f"PageRank with r = {r}:")
    print(page_rank)

PageRank with r = 0.1:
[4.39778657e+74 2.00801960e+74 1.14082036e+74 8.23438956e+73
 7.02336840e+73 6.47119797e+73 6.06203789e+73 5.52871551e+73
 4.62650623e+73 2.99436315e+73 9.22449818e+74]
PageRank with r = 0.01:
[6.46777225e+90 2.62325926e+90 1.09940320e+90 4.94756734e+89
 2.53776013e+89 1.55944577e+89 1.13235188e+89 8.96757040e+88
 6.91752897e+88 4.22671927e+88 1.58665152e+91]
PageRank with r = 0.001:
[2.95829112e+92 1.18496831e+92 4.76280122e+91 1.93028956e+91
 7.97646312e+90 3.43842296e+90 1.60535642e+90 8.40195669e+89
 4.79996446e+89 2.45262857e+89 7.38251179e+92]
PageRank with r = 0.0001:
[4.33639409e+92 1.73479740e+92 6.94250198e+91 2.78060546e+91
 1.11584107e+91 4.49728490e+90 1.82859873e+90 7.53748415e+89
 3.11387909e+89 1.13690681e+89 1.08403549e+93]


In [19]:

N = 1000
p = 0.4
r = 0.1

# Create the transition matrix with restart probability
def crt_trns_mat(N, p, r):
    P =matrix(N)
    # Add restart probability
    P = (1 - r) * P + r / (N + 1)
    return P

# Create the transition matrix
P = crt_trns_mat(N, p, r)

# Compute the stationary distribution using eig
e_val, e_vec = np.linalg.eig(P.T)
stat_dbt = np.real(e_vec[:, np.argmax(np.abs(e_val))])
stat_dbt /= np.sum(stat_dbt)

print("PageRank for N = 1000 and r = 0.1:")
print(stat_dbt)

PageRank for N = 1000 and r = 0.1:
[1.30500936e-01 5.23659967e-02 2.11781301e-02 ... 2.94724296e-04
 1.84348912e-04 3.24840181e-01]


In [8]:

from scipy.sparse import coo_matrix

def powerP(x, aPt, r=0.1, maxn=1000, tol=1e-10):
    for n in range(maxn):
        # Calculate (Pt)^n * x using the given aPt function
        Ptx = aPt(x)

        # Calculate the next iteration of x with the restart probability
        next_x = (1 - r) * Ptx + r / len(x)

        # Check for convergence
        if np.linalg.norm(next_x - x) < tol:
            return next_x

        x = next_x

    # Return the result after maxn iterations (may not have converged)
    return x

# Parameters
N = 100000
p = 0.4
r = 0.1

# Create the transition matrix with restart probability (sparse matrix)
data = []
row_ind = []
col_ind = []

# Handle the boundary cases separately to avoid negative column indices
data.extend([1 - r, p])
row_ind.extend([0, 0])
col_ind.extend([0, 1])

for i in range(1, N):
    data.extend([p, 1 - p, 1 - r, r])
    row_ind.extend([i, i, i, i])
    col_ind.extend([i + 1, i - 1, i, i])

data.extend([1 - p, 1 - r])
row_ind.extend([N, N])
col_ind.extend([N - 1, N])

shape = (N + 1, N + 1)

P_sparse = coo_matrix((data, (row_ind, col_ind)), shape=shape)

# Initial probability distribution (start with a random distribution)
initial_distribution = np.random.rand(N + 1)
initial_distribution /= np.sum(initial_distribution)

# Define a function for applying Pt to a vector (sparse matrix multiplication)
def apply_Pt(x):
    return P_sparse.T.dot(x)

# Compute the PageRank using the powerP function
page_rank = powerP(initial_distribution, apply_Pt, r=r)

print("PageRank for r = 0.1, N = 100000:")
print(page_rank)


PageRank for r = 0.1, N = 100000:
[5.44845133e+249 9.98941636e+249 1.30178778e+250 ... 6.46435488e+243
 3.42839665e+243 1.27326487e+243]
